In [130]:
import pandas as pd
from ipywidgets import Video, Image 
!jupyter nbextension list
# (you need to see both jupyter-js-widgets/extension and jupyter-video/extension enabled and validated below)
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install Jupyter-Video_Widget
# !jupyter nbextension enable --py --sys-prefix jpy_video

Known nbextensions:
  config dir: /root/anaconda3/etc/jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
      jupyter-video/extension  enabled 
      - Validating: OK


In [119]:
gt = pd.read_csv("ground_truth.csv",header=0,index_col=1)

In [88]:
gt.head()

,video-UID,labeler,class
clip_title,,,
5yo_vs_Mickey_slice0_slice0.mp4,10000,Alex,0
5yo_vs_Mickey_slice10_slice0.mp4,10001,Alex,0
5yo_vs_Mickey_slice15_slice0.mp4,10002,Alex,0
5yo_vs_Mickey_slice20.mp4,10003,Ahsen,0
5yo_vs_Mickey_slice5.mp4,10004,Jen,0


In [120]:
df = pd.read_csv("eval/punch101_test_preds.txt",header=0)
df['filename'] = df['Path'].apply(lambda x: x.split("/")[1].replace("_scaled",".mp4"))
df.drop('Path',inplace=True,axis=1)
df.set_index('filename',inplace=True)

In [109]:
df.head()

,RGB_Preds,Flow_Preds,Combined_Preds,Truth
filename,,,,
heavy_bag_drills_slice105.mp4,1,1,1,1
NV_630.mp4,0,0,0,0
V_980.mp4,3,0,0,0
streetfight5_slice680.mp4,0,0,0,0
SouthAmericanDrumline_slice35.mp4,0,0,0,0


In [110]:
df.shape

(1060, 4)

In [121]:
df = df.join(gt).reset_index().set_index('video-UID')
df.rename(columns={'filename':'clip_title'},inplace=True)

In [99]:
df.shape

(1060, 7)

## Jen is concerned that the "Truth" column coming from Lance's 'punch101_test_preds.txt' file does not always match the "class" column coming from our 'ground_truth.csv' master labeled datasest.

In [131]:
df[df['Truth'] != df['class']].sample(10)

,clip_title,RGB_Preds,Flow_Preds,Combined_Preds,Truth,labeler,class
video-UID,,,,,,,
14907,streetfight3_slice525.mp4,3,3,3,4,Jen,2
11887,V_345.mp4,0,0,0,3,Alex,1
11845,V_306_slice0.mp4,3,3,3,3,Jen,1
11995,V_438.mp4,3,3,3,3,Jen,1
12279,V_695.mp4,0,3,3,3,Lance,1
11987,V_430.mp4,3,3,3,3,Alex,1
2282,V_755.mp4,3,0,3,3,Jen,1
13118,fast_punch_drill_slice160.mp4,1,1,1,1,Ahsen,3
2248,V_724.mp4,0,0,0,3,Alex,1


In [139]:
# Play a few examples from above "Truth"/"class" mismatch subset
# Jen is pretty sure "class" is what we should be measuring against
# clip = 'streetfight3_slice525.mp4'
# clip = 'V_345.mp4'
clip = 'fast_punch_drill_slice160.mp4'

In [140]:
show = Video.from_file('/root/vids/' + clip)
show

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x0f.8mdat\x00\x00\…